This section aims to try different approaches. Some of the project's choices might be hindering the model's performance such as brand-model grouped imputation.

#

In [110]:
# Locate project root dir, enable package imports from src/
import sys
from pathlib import Path

PROJ_ROOT = Path().resolve().parents[1]
sys.path.append(str(PROJ_ROOT / "src"))
print(PROJ_ROOT)

/Users/alexandre/Documents/GitHub/price_predictor


In [ ]:
import sys
import os
import pandas as pd
import numpy as np
from math import ceil, floor
import json

# Load standardized paths, ml utilities and custom transformer
from project_utils.paths import (
    RAW_DATA_DIR,
    TEST_PREDICTIONS_DIR,
)
import project_utils.ml_utils as ml_utils
from open_ended_utils import feature_engineer, simpler_target_encoder, validity_cleaners

# Holdout method
from sklearn.model_selection import train_test_split

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)


from sklearn.base import clone

from sklearn.preprocessing import (
    RobustScaler,
)

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import (
    HistGradientBoostingRegressor,
)

from sklearn.compose import make_column_selector


In [ ]:
# Adds the project root to the path so 'Helpers' can be found
# '..' goes up one level from 'Notebooks/' to 'project-root/'




In [ ]:
df_train = pd.read_csv(os.path.join(RAW_DATA_DIR, "train.csv"), index_col="carID")
X_test = pd.read_csv(os.path.join(RAW_DATA_DIR, "test.csv"), index_col="carID")

X = df_train.drop("price", axis=1).copy()
y = np.log1p(df_train["price"].copy())

# Performing the holdout methods
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=0)

# Experimenting new pipeline

This notebook focuses exclusively on the HistGradientBoostingRegressor, an ensemble tree-based method that demonstrated the strongest predictive performance across all models evaluated in this project. The primary objective of this section is to present a fully integrated, end-to-end pipeline that consolidates all data preprocessing, feature engineering, and modeling steps into a single, coherent workflow.

In contrast to earlier analyses, where hyperparameter optimization was restricted to the estimator level, this section adopts a more comprehensive strategy. A randomized hyperparameter search is conducted across all stages of the pipeline, encompassing not only model-specific parameters but also those associated with data validation, cleaning, and feature transformation. This holistic approach allows for a more thorough exploration of interactions between preprocessing decisions and model behavior.

The complete pipeline is structured sequentially as follows: data validity checking and cleaning, feature engineering, imputation, encoding, scaling, feature selection, and final model estimation. By integrating these components within a unified framework, this approach enhances both the robustness and reproducibility of the modeling process, while providing a systematic basis for evaluating the combined impact of preprocessing and modeling choices on overall performance.

In [ ]:
# --- Dictionary of valid categorical features --- #
cleaned_dict_dir = os.path.join(
    PROJ_ROOT, "src", "project_utils", "valid_categories.json"
)
with open(cleaned_dict_dir, "r") as f:
    valid_categories = json.load(f)

# --- Dictionary of valid numerical features --- #
numeric_ranges = {
    "year": (0, 2020),
    "mileage": (0, None),
    "engineSize": (0, None),
    "mpg": (0, 300),
    "tax": (0, None),
    "previousOwners": (0, None),
}

In [115]:
best_params = {
    "cleaner_cat_replace_with": np.nan,
    "cleaner_min_similarity": 0.85,
    "cleaner_numeric_policy": "wipe",
    "imputer": "passthrough",
    "model_early_stopping": True,
    "model_l2_regularization": 0.017619542016522542,
    "model_learning_rate": 0.03833834069078183,
    "model_max_depth": 30,
    "model_max_iter": 2495,
    "model_max_leaf_nodes": 4540,
    "model_min_samples_leaf": 44,
    "model_random_state": 99,
}

## HGBR

In [116]:
pipeline = Pipeline(
    steps=[
        (
            "cleaner",
            validity_cleaners.FullValidityCleaner(
                valids=valid_categories,
                ranges=numeric_ranges,
            ),
        ),
        (
            "feature_engineer",
            feature_engineer.FeatureEngineer(reference_year=X_train["year"].max()),
        ),
        ("imputer", "passthrough"),
        (
            "encoder",
            ColumnTransformer(
                transformers=[
                    (
                        "num",
                        "passthrough",
                        make_column_selector(dtype_include=np.number),
                    ),
                    (
                        "cat",
                        simpler_target_encoder.MeanTargetEncoder(),
                        make_column_selector(dtype_exclude=np.number),
                    ),
                ],
                remainder="drop",
            ),
        ),
        ("scaler", RobustScaler()),
        ("feature_selector", "passthrough"),
        ("model", HistGradientBoostingRegressor()),
    ]
)


# --- Raw best params (e.g., loaded from JSON / previous output) --- #
raw_best_params = {
    "cleaner_cat_replace_with": np.nan,
    "cleaner_min_similarity": 0.85,
    "cleaner_numeric_policy": "wipe",
    "model_early_stopping": True,
    "model_l2_regularization": 0.017619542016522542,
    "model_learning_rate": 0.03833834069078183,
    "model_max_depth": 30,
    "model_max_iter": 2495,
    "model_max_leaf_nodes": 4540,
    "model_min_samples_leaf": 44,
    "model_random_state": 99,
}

# --- Convert to pipeline parameter names --- #
pipeline_best_params = {
    "cleaner__cat_replace_with": raw_best_params["cleaner_cat_replace_with"],
    "cleaner__min_similarity": raw_best_params["cleaner_min_similarity"],
    "cleaner__numeric_policy": raw_best_params["cleaner_numeric_policy"],
    "model__early_stopping": raw_best_params["model_early_stopping"],
    "model__l2_regularization": raw_best_params["model_l2_regularization"],
    "model__learning_rate": raw_best_params["model_learning_rate"],
    "model__max_depth": raw_best_params["model_max_depth"],
    "model__max_iter": raw_best_params["model_max_iter"],
    "model__max_leaf_nodes": raw_best_params["model_max_leaf_nodes"],
    "model__min_samples_leaf": raw_best_params["model_min_samples_leaf"],
    "model__random_state": raw_best_params["model_random_state"],
}

pipeline.set_params(**pipeline_best_params)
hgbr = pipeline.fit(X_train, y_train)
hgbr


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('cleaner', ...), ('feature_engineer', ...), ...]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,valids,"{'categorical_valid_values': {'Brand': ['toyota', 'mercedes', ...], 'fuelTypes': ['petrol', 'diesel', ...], 'model': ['golf', 'passat', ...], 'transmission': ['manual', 'semi-auto', ...]}, 'numeric_ranges': {'engineSize': {'max': None, 'min': 0}, 'mileage': {'max': None, 'min': 0}, 'mpg': {'max': 300, 'min': 0}, 'previousOwners': {'max': None, 'min': 0}, ...}}"
,ranges,"{'engineSize': (0, ...), 'mileage': (0, ...), 'mpg': (0, ...), 'previousOwners': (0, ...), ...}"
,numeric_policy,'wipe'
,min_similarity,0.85
,cat_replace_with,nan
,add_invalid_flag,True
,normalize_strings,True


In [117]:
# Compute evaluation metrics for the training and validation sets
y_train = np.expm1(y_train)
y_val = np.expm1(y_val)
# Training set
train_predictions = np.expm1(pipeline.predict(X_train))
print("Training set metrics:")
metrics_train = ml_utils.linear_evaluation_metrics(
    y_train, train_predictions, verbose=True
)
# Validation set
validation_predictions = np.expm1(pipeline.predict(X_val))
print("Validation set metrics:")
metrics_val = ml_utils.linear_evaluation_metrics(
    y_val, validation_predictions, verbose=True
)

Training set metrics:
R²: 0.969, MAE: 935.48, MAPE: 5.59%, RMSE: 1711.57
-------------------------------------------------------
Validation set metrics:
R²: 0.944, MAE: 1337.64, MAPE: 7.88%, RMSE: 2391.40
-------------------------------------------------------


In [118]:
# -- Export X_test predictions --#
pred_y_test = pipeline.predict(X_test)
# Convert to Series
test_predictions_series = pd.Series(data=pred_y_test, name="price", index=X_test.index)

# Export CSV file
test_predictions_series.to_csv(
    os.path.join(TEST_PREDICTIONS_DIR, "open_ended_test_predictions.csv")
)

This approach integrates data cleaning directly into the modeling pipeline while simultaneously performing a randomized search over the full set of hyperparameters. This strategy proved effective, yielding substantial improvements in overall model performance. Moreover, the retention of variables previously identified as non-informative by Elastic Net, together with the inclusion of newly engineered features, suggests that multiple complementary methodological approaches can meaningfully contribute to performance gains.

Overall, this section demonstrates the existence of several viable avenues through which model performance can be enhanced, highlighting the value of combining robust preprocessing, feature engineering, and flexible model selection strategies.